In [73]:
import numpy as np
import pandas as pd
import luxpy as lp
from scipy.interpolate import CubicSpline
np.set_printoptions(suppress=False) 

### 1. import IES files

In [241]:
# pip install luxpy

In [242]:
ies = lp.toolboxes.iolidfiles.read_lamp_data('ies/4.ies')

In [243]:
ies.keys()

dict_keys(['datasource', 'name', 'version', 'lamps_num', 'lumens_per_lamp', 'candela_mult', 'v_angles_num', 'h_angles_num', 'photometric_type', 'units_type', 'width', 'length', 'height', 'ballast_factor', 'future_use', 'input_watts', 'v_angs', 'h_angs', 'lamp_cone_type', 'lamp_h_type', 'candela_values', 'candela_2d', 'v_same', 'h_same', 'intensity', 'map', 'norm_angs', 'Isym', 'theta', 'phi', 'values', 'Iv0'])

In [244]:
print(' luminaire width: ', ies['width'], '\n', 
      'luminaire length: ', ies['length'],'\n', 
      'luminaire height: ', ies['height'],'\n',
     'luminaire wattage: ', ies['input_watts'], '\n', 
      'candela multiplier: ', ies['candela_mult'])

 luminaire width:  -1.041667 
 luminaire length:  -1.041667 
 luminaire height:  0.114833 
 luminaire wattage:  88.7 
 candela multiplier:  0.9687


Key `candela_mult`, `v_angs`, `h_angs`, `candela_values` were used below.

#### There are three types of Luminiare testing methods - Indoor, Roadway and Floodlight 


#### a. This is the coordinate system when the fixture is tested as indoor / roadway luminiare

<img src="img/1_0.png" width="700">

#### b. This is the coordinate system when the fixture is tested as floodlight luminiare

<img src="img/1_2.png" width="450">

#### The following codes only apply to condition `a`, when the fixture is tested as indoor / roadway luminiare

#### 1.1 raw Intensity (candela) values

In [245]:
n_h_angles = len(ies['h_angs'])
n_v_angles = len(ies['v_angs'])

# create numpy array and insert intensity values(multiplied by candela_mult)
cd = np.zeros((n_h_angles, n_v_angles))

for i in range(n_h_angles):
    end_angles = n_v_angles*(i+1)
    cd[i] = ies['candela_values'][i*n_v_angles:end_angles]*ies['candela_mult']

In [246]:
cd.shape

(9, 73)

#### 1.2 broadcast intensity values to full 0-360° horizontal plane

* `Quadrilateral symmetric` horizontal testing angles within [0-90°]
* `Bilateral symmetric` horizontal testing angles angles within [0-180°]
* `Axial symmetric` horizontal testing angles angle = 0°
* `Asymetric` A complete set of horizontal testing angles from 0-360° (may or may not include 360°)

In [247]:
ies['h_angs']

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ])

In [248]:
def broadcast(cd, h_angs):
    
    if max(h_angs)==360: # asymmetric, 360° is included
        full_h_angs = h_angs
        cd_0_360 = cd
        
    elif max(h_angs) > 180: # asymmetric, 360° not included
        full_h_angs = np.append(h_angs,360) # add 360°
        cd_0_360 = np.vstack((cd, cd[0])) # copy intensity data @0° to 360°
    
    elif max(h_angs) == 90: # quadrilateral symmetric
        h_angs_1 = 180 - h_angs[0:-1] # angle for 90° - 180° plane
        h_angs_2 = 180 + h_angs[1:] # angle for 0° - 180° plane
        h_angs_3 = 360 - h_angs[0:-1] # angle for 180° - 360° plane
        full_h_angs = np.sort(np.concatenate((h_angs, h_angs_1, h_angs_2, h_angs_3))) # full horizontal angles
        cd_90_180 = cd[:-1][::-1]  # intensity value for 90° - 180° plane
        cd_0_180 = np.vstack((cd, cd_90_180))  # intensity value for 0° - 180° plane
        cd_180_360 = cd_0_180[:-1][::-1]   # intensity value for 180° - 360° plane
        cd_0_360 = np.vstack((cd_0_180, cd_180_360))  # intensity value for 0° - 360° plane
        
    elif max(h_angs) == 180: # bilateral symmetric
        h_angs_1 = 360 - h_angs[:-1]
        full_h_angs = np.sort(np.concatenate((h_angs, h_angs_1))) # full horizontal angles
        cd_180_360 = cd[:-1][::-1] # intensity value for 180° - 360° plane
        cd_0_360 = np.vstack((cd, cd_180_360))  # intensity value for 0° - 360° plane
        
    elif max(h_angs) == 0: # axial symmetric
        full_h_angs = np.arange(0,360+1, 22.5) # standard indoor luminaire horizontal testing intervals
        cd_0_360 = np.repeat([cd], len(full_h_angs), axis=0) # intensity value for 0° - 360° plane
        
    return cd_0_360, full_h_angs
    

In [249]:
full_cd, full_h_angs = broadcast(cd, ies['h_angs'])

In [250]:
display(full_cd, full_h_angs)

array([[2540.9001, 2559.3054, 2545.7436, ...,    0.    ,    0.    ,
           0.    ],
       [2540.9001, 2515.7139, 2543.8062, ...,    0.    ,    0.    ,
           0.    ],
       [2540.9001, 2513.7765, 2542.8375, ...,    0.    ,    0.    ,
           0.    ],
       ...,
       [2540.9001, 2513.7765, 2542.8375, ...,    0.    ,    0.    ,
           0.    ],
       [2540.9001, 2515.7139, 2543.8062, ...,    0.    ,    0.    ,
           0.    ],
       [2540.9001, 2559.3054, 2545.7436, ...,    0.    ,    0.    ,
           0.    ]])

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5, 360. ])

-------------------------------------------------------------------------------

#### 1.3 Interpolate intensity value if raw data is not evenly distributed, delete duplicated data at horizontal 360° 

In [251]:
def interpolate(cd, h_angs, v_angs):
    
    if all(np.diff(h_angs)==np.diff(h_angs)[0]): # horizontal angles have equal intervals
        cd_inter = cd[:-1].transpose() # delete data @360°, transpose data for future use
        h_angs_inter = h_angs
        
    else: # horizontal angles have unequal intervals
        min_step = np.diff(h_angs).min() # find the minimum interval
        cd_T = cd.transpose() # transpose data for calculation
        
        if np.sum(np.diff(ies['h_angs'])%min_step) == 0:
            h_angs_inter = np.arange(0, 361, step=min_step) # create an array of angles with equal interval            
            cd_inter = np.zeros((len(v_angs), len(h_angs_inter))) 
            for i in range(len(v_angs)):
                cd_inter[i] = np.interp(h_angs_inter, h_angs, cd_T[i]) # linear interpolation
                
        else:
            h_angs_inter = np.arange(0, 361, step=1) # create an array of angles with equal interval
            cd_inter = np.zeros((len(v_angs), len(h_angs_inter))) 
            for i in range(len(v_angs)):
                f = CubicSpline(h_angs, cd_T[i], bc_type='natural') # Cubic spline interpolation
                cd_inter[i] = f(h_angs_inter)
            
        cd_inter = np.delete(cd_inter, -1, axis=1) # delete data @360°
        
    return cd_inter, h_angs_inter

In [252]:
cd_inter, h_angs_inter = interpolate(full_cd, full_h_angs, ies['v_angs'])

In [253]:
cd_inter.shape

(73, 16)

In [254]:
cd_inter

array([[2540.9001, 2540.9001, 2540.9001, ..., 2540.9001, 2540.9001,
        2540.9001],
       [2559.3054, 2515.7139, 2513.7765, ..., 2558.3367, 2513.7765,
        2515.7139],
       [2545.7436, 2543.8062, 2542.8375, ..., 2542.8375, 2542.8375,
        2543.8062],
       ...,
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ],
       [   0.    ,    0.    ,    0.    , ...,    0.    ,    0.    ,
           0.    ]])

### 2. Zonal Lumen calculation

<img src="img/2_1.png" width="300">

#### Flux in a conic solid angle is given by: 
$$
  \Phi_N = 2\pi I_{\theta_N} (cos{\theta_N} - cos{\theta_{N+1}}  )
$$

#### 2.1 Caculate zonal constant for each vertical angle

In [255]:
v_angs_1 = ies['v_angs'][:-1] 
v_angs_2 = ies['v_angs'][1:]
c = zip(v_angs_1, v_angs_2)

zonal_constant = []

for a, b in c:
    zonal_constant.append(2*np.pi*(np.cos(a*np.pi/180) - np.cos(b*np.pi/180)))

#### 2.2 Caculate average instensity value(cd) between two vertical angle zones

In [256]:
avg_cd = [(np.mean(cd_inter[i])+np.mean(cd_inter[i+1]))/2 for i in range(len(ies['v_angs'])-1)]

In [257]:
# create a dataframe with vertical angles, average instensity and zonal constant data

df = pd.DataFrame({'v_angle_1': ies['v_angs'][:-1],
                   'v_angle_2': ies['v_angs'][1:],
                   'average_insensity': avg_cd, 
                   'zonal_constant': np.array(zonal_constant)})

#### 2.3 Caculate zonal lumen for each vertical angle zone

In [258]:
df['zonal_lumen'] = np.round(df['average_insensity']*df['zonal_constant'],2)

In [259]:
df

,v_angle_1,v_angle_2,average_insensity,zonal_constant,zonal_lumen
0,0.0,2.5,2533.332131,0.005980,15.15
1,2.5,5.0,2532.030441,0.017929,45.40
2,5.0,7.5,2524.795463,0.029844,75.35
3,7.5,10.0,2482.142391,0.041702,103.51
4,10.0,12.5,2435.584247,0.053481,130.26
...,...,...,...,...,...
67,167.5,170.0,0.000000,0.053481,0.00
68,170.0,172.5,0.000000,0.041702,0.00
69,172.5,175.0,0.000000,0.029844,0.00
70,175.0,177.5,0.000000,0.017929,0.00


#### 2.4 convert zonal lumen to standard 18 zones (10° interval)

In [260]:
label = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130','130-140', '140-150', '150-160', '160-170', '170-180' ]
df['bin'] = pd.cut(df['v_angle_2'], bins=np.arange(0,181,10), labels=label)


In [261]:
zl = pd.DataFrame({'zonal_lumen': df.groupby('bin').sum()['zonal_lumen'].round(2),
                  'zone_number': np.arange(1,19,1)})

C:\Users\guym\AppData\Local\Temp/ipykernel_1376/2851976775.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  zl = pd.DataFrame({'zonal_lumen': df.groupby('bin').sum()['zonal_lumen'].round(2),


In [262]:
zl

,zonal_lumen,zone_number
bin,,
0-10,239.41,1
10-20,664.74,2
20-30,938.53,3
30-40,1024.22,4
40-50,931.54,5
50-60,763.54,6
60-70,581.73,7
70-80,500.29,8
80-90,175.30,9


In [263]:
# create a dict of zonal lumens, with zone number 1 to 9

zl_dict = dict(zip(zl['zone_number'][:9], zl['zonal_lumen'][:9]))

In [264]:
zl_dict

{1: 239.41,
 2: 664.74,
 3: 938.53,
 4: 1024.22,
 5: 931.54,
 6: 763.54,
 7: 581.73,
 8: 500.29,
 9: 175.3}

### 3. Determine the additional flux functions

In [265]:
# total lumens
flux = zl['zonal_lumen'].sum()

# lumen downwards, zone number 1 to 9
flux_down = zl['zonal_lumen'][0:9].sum()

# proportion of lumens in a downward direction
percent_down = flux_down/flux

# proportion of lumens in a upward direction
percent_up = 1 - percent_down

In [266]:
print(flux, flux_down,percent_down, percent_up)

5821.780000000001 5819.3 0.9995740134460593 0.00042598655394066043


### 4. Determine the direct ratio

* Direct Ratio ($\begin{aligned}D_{RCR}\end{aligned}$) is the fraction of the downward luminaire flux that reaches the work plain directly.  
* $\begin{aligned}K_{RCR, N}\end{aligned}$ is called a zonal multiplier and is a function of both cavity ratio, denoted by RCR and zone N.
* RCR is the room cavity ratio; between 1 and 10, inclusive. 
* RCR =0 is a special case, we will revisit this case later.

$$
  D_{RCR} = \frac{1}{\Phi_{down} }\sum_{n=1}^9 {k_{RCR, N}}{\Phi_N}
$$

$$
  K_{RCR,N} = e^{-A * {RCR^B}} 
$$

where A and B are constants for each flux zone

<img src="img/4_1.png" width="300">

In [267]:
# dict for zonal multiplier

a = [0, 0.041, 0.070, 0.1, 0.136, 0.19,0.315, 0.64, 2.1]
b = [0, 0.98, 1.05, 1.12, 1.16, 1.25, 1.25, 1.25, 0.8]
zone = np.arange(1,10)

zonal_mul_constant = dict(zip(zone, zip(a, b)))

display(zonal_mul_constant)

{1: (0, 0),
 2: (0.041, 0.98),
 3: (0.07, 1.05),
 4: (0.1, 1.12),
 5: (0.136, 1.16),
 6: (0.19, 1.25),
 7: (0.315, 1.25),
 8: (0.64, 1.25),
 9: (2.1, 0.8)}

In [268]:
# calculate direct ratio for each RCR value (1 to 10)

direct_ratio = {}

for rcr in range(1,11):
    flux_sum = 0 
    for zone in range(1,10):
        flux_sum += np.exp(-1 * zonal_mul_constant[zone][0] * np.power(rcr, zonal_mul_constant[zone][1]))*zl_dict[zone]
    direct_ratio[rcr] = np.round(flux_sum/flux_down,3)
        

In [269]:
direct_ratio

{1: 0.831,
 2: 0.696,
 3: 0.593,
 4: 0.513,
 5: 0.449,
 6: 0.398,
 7: 0.356,
 8: 0.321,
 9: 0.291,
 10: 0.266}

### 5. Determin parameters $\begin{aligned}C_1, C_2, C_3, C_0 \end{aligned}$

$$
  C_1 = \frac{(1 - {\rho_W})(1-f^2_{CC{\rightarrow}FC})RCR}{2.5{\rho_W}(1-f^2_{CC{\rightarrow}FC}) + RCR {\bullet}f_ {CC{\rightarrow}FC}(1-{\rho_W})}
$$

$$
  C_2 = \frac{(1 - {\rho_{CC}})(1+f_{CC{\rightarrow}FC})}{1 + {\rho_{CC}} f_{CC{\rightarrow}FC}}
$$

$$
  C_3 = \frac{(1 - {\rho_{FC}})(1+f_{CC{\rightarrow}FC})}{1 + {\rho_{FC}} f_{CC{\rightarrow}FC}}
$$

$$
  C_0 = C_1 + C_2 + C_3
$$

* $\begin{aligned}{\rho_{CC}}\end{aligned}$ is the ceiling cavity reflectance  
* $\begin{aligned}{\rho_{W}}\end{aligned}$ is the wall reflectance
* $\begin{aligned}{\rho_{FC}}\end{aligned}$ is the floor cavity reflectance, which is taken as 0.2 for standard CU tables
* $\begin{aligned}{f_{CC{\rightarrow}FC}}\end{aligned}$ is the form factor from the ceiling cavity to the floor cativity, shown in the table below


<img src="img/5_1.png" width="400">

In [270]:
# form factor for RCR 1 to 10
form_factor_value = [0.827, 0.689, 0.579, 0.489, 0.415, 0.355, 0.306, 0.265, 0.231, 0.202]

form_factor = dict(zip(np.arange(1,11), form_factor_value))

form_factor

{1: 0.827,
 2: 0.689,
 3: 0.579,
 4: 0.489,
 5: 0.415,
 6: 0.355,
 7: 0.306,
 8: 0.265,
 9: 0.231,
 10: 0.202}

### 6. Determine the CU for each combination of reflectance and RCR


<img src="img/6_1.png" width="600">

#### This is a sample CU table generated by Photometric Toolbox
* $\begin{aligned}{\rho_{CC}}\end{aligned}$ value: 0.8/0.7/0.5/0.3/0.1/0
* $\begin{aligned}{\rho_{w}}\end{aligned}$ value: 0.7/0.5/0.3/0.1/0
* $\begin{aligned}{\rho_{FC}}\end{aligned}$ value: 0.2

In [271]:
reflectance_pair_cw = [[0.8,0.7], [0.8,0.5], [0.8,0.3], [0.8,0.1], [0.7, 0.7], [0.7,0.5], [0.7,0.3], [0.7,0.1], [0.5,0.5], [0.5,0.3], [0.5,0.1], [0.3,0.5], [0.3,0.3], [0.3,0.1],[0.1,0.5], [0.1,0.3],[0.1,0.1],[0,0]]

#### 6.1 CU for RCR 1 to 10

In [272]:
cu = {}
for rcr in range(1,11):
    for r in reflectance_pair_cw:
        c1 = (1 - r[1])*(1 - np.power(form_factor[rcr], 2))*rcr/(2.5*r[1]*(1 - np.power(form_factor[rcr], 2)) + rcr*form_factor[rcr]*(1 - r[1]))
        c2 = (1 - r[0])*(1 + form_factor[rcr])/(1 + r[0]*form_factor[rcr])
        c3 = (1 - 0.2)*(1 + form_factor[rcr])/(1+0.2*form_factor[rcr])
        c0 = c1 + c2 + c3
        cu1 = 2.5*r[1]*c1*c3*(1 - direct_ratio[rcr])*percent_down/(rcr*(1 - r[1])*(1 - 0.2)*c0)
        cu2 = r[0]*c2*c3*percent_up/((1 - r[0])*(1 - 0.2)*c0)
        cu3 = (1 - 0.2*c3*(c1 + c2)/((1 - 0.2)*c0))*(direct_ratio[rcr]*percent_down/(1 - 0.2))
        cu[(rcr,r[0], r[1])] = np.round((cu1 + cu2 + cu3)*100,0)
        

#### 6.2 CU for RCR = 0

In [273]:
for r in reflectance_pair_cw:
    cu[(0,r[0], r[1])] = np.round((percent_down + r[0]*percent_up)/(1 - r[0]*0.2)*100, 0)

In [274]:
cu

{(1, 0.8, 0.7): 108.0,
 (1, 0.8, 0.5): 104.0,
 (1, 0.8, 0.3): 99.0,
 (1, 0.8, 0.1): 95.0,
 (1, 0.7, 0.7): 106.0,
 (1, 0.7, 0.5): 101.0,
 (1, 0.7, 0.3): 97.0,
 (1, 0.7, 0.1): 94.0,
 (1, 0.5, 0.5): 97.0,
 (1, 0.5, 0.3): 94.0,
 (1, 0.5, 0.1): 91.0,
 (1, 0.3, 0.5): 93.0,
 (1, 0.3, 0.3): 90.0,
 (1, 0.3, 0.1): 88.0,
 (1, 0.1, 0.5): 89.0,
 (1, 0.1, 0.3): 87.0,
 (1, 0.1, 0.1): 85.0,
 (1, 0, 0): 83.0,
 (2, 0.8, 0.7): 99.0,
 (2, 0.8, 0.5): 91.0,
 (2, 0.8, 0.3): 84.0,
 (2, 0.8, 0.1): 78.0,
 (2, 0.7, 0.7): 96.0,
 (2, 0.7, 0.5): 89.0,
 (2, 0.7, 0.3): 82.0,
 (2, 0.7, 0.1): 77.0,
 (2, 0.5, 0.5): 85.0,
 (2, 0.5, 0.3): 80.0,
 (2, 0.5, 0.1): 75.0,
 (2, 0.3, 0.5): 82.0,
 (2, 0.3, 0.3): 77.0,
 (2, 0.3, 0.1): 73.0,
 (2, 0.1, 0.5): 79.0,
 (2, 0.1, 0.3): 75.0,
 (2, 0.1, 0.1): 72.0,
 (2, 0, 0): 70.0,
 (3, 0.8, 0.7): 90.0,
 (3, 0.8, 0.5): 80.0,
 (3, 0.8, 0.3): 72.0,
 (3, 0.8, 0.1): 65.0,
 (3, 0.7, 0.7): 88.0,
 (3, 0.7, 0.5): 78.0,
 (3, 0.7, 0.3): 71.0,
 (3, 0.7, 0.1): 65.0,
 (3, 0.5, 0.5): 75.0,
 (3, 0.5, 0.3)

#### CU dictionary key:  (RCR, ceiling cavity reflectance, wall reflectance)